In [1]:
import gurobipy as gp
import pandas as pd
from code_map import data_handling
import numpy as np
import calendar 
from datetime import datetime
import pytz
import openpyxl
import os
import plotly.graph_objects as go
import plotly.express as px

In [3]:
weather_data = pd.read_csv('../master-data/weather_data/weather_5_meters.csv', sep=';')

In [37]:
len(weather_data)

47571

In [35]:
def preprocess_weather_data(df : pd.DataFrame):
    #df.drop(df.loc[df["Navn"] ==  'Data er gyldig per 06.12.2023 (CC BY 4.0), Meteorologisk institutt (MET)'], inplace=True)
    df.rename(columns={'Tid(norsk normaltid)' : 'Time (Local)', \
                       "Navn" : "station_name", \
                       "Stasjon" : "station", \
                       "Lufttemperatur" : "air_temp", \
                       "Vindretning" : "wind_direction" , \
                       "Middelvind" : "wind_strength" , \
                       "Nedbør (1 t)" : "precipitation"}, inplace=True)
    print(df.columns)
    # get this format : 01.01.2022 01:00
    format = '%d.%m.%Y %H:%M'
    df['Time (Local)'] = pd.to_datetime(df['Time (Local)'], format= format, utc=True)
    df['Time (Local)'] = df['Time (Local)'].dt.tz_convert('Europe/Oslo')
    df.sort_values(by=['Time (Local)'], inplace=True)
    df.reset_index(drop=True, inplace= True)
    df.drop(['level_0', 'index'], axis=1, inplace=True)
    return df

In [36]:
upd_w_df = preprocess_weather_data(weather_data)

Index(['level_0', 'index', 'station_name', 'station', 'Time (Local)',
       'air_temp', 'wind_direction', 'wind_strength', 'precipitation'],
      dtype='object')


In [30]:
upd_w_df.head()

,level_0,index,station_name,station,Time (Local),air_temp,wind_direction,wind_strength,precipitation
0,0,0,Bergen - Florida,SN50540,2022-01-01 02:00:00+01:00,"2,8",279,"1,1",0
1,1,33161,Trondheim - Risvollan,SN68230,2022-01-01 02:00:00+01:00,"-0,3",-,-,0
2,2,33162,Trondheim - Risvollan,SN68230,2022-01-01 03:00:00+01:00,"-1,4",-,-,0
3,3,1,Bergen - Florida,SN50540,2022-01-01 03:00:00+01:00,3,92,"0,6",0
4,4,2,Bergen - Florida,SN50540,2022-01-01 04:00:00+01:00,"2,8",90,"1,8",0


In [38]:
weather_data["Time (Local)"].unique()

<DatetimeArray>
['2022-01-01 02:00:00+01:00', '2022-01-01 03:00:00+01:00',
 '2022-01-01 04:00:00+01:00', '2022-01-01 05:00:00+01:00',
 '2022-01-01 06:00:00+01:00', '2022-01-01 07:00:00+01:00',
 '2022-01-01 08:00:00+01:00', '2022-01-01 09:00:00+01:00',
 '2022-01-01 10:00:00+01:00', '2022-01-01 11:00:00+01:00',
 ...
 '2023-12-06 03:00:00+01:00', '2023-12-06 04:00:00+01:00',
 '2023-12-06 05:00:00+01:00', '2023-12-06 06:00:00+01:00',
 '2023-12-06 07:00:00+01:00', '2023-12-06 08:00:00+01:00',
 '2023-12-06 09:00:00+01:00', '2023-12-06 10:00:00+01:00',
 '2023-12-06 11:00:00+01:00',                       'NaT']
Length: 16907, dtype: datetime64[ns, Europe/Oslo]

In [10]:
weather_data.loc[weather_data["Navn"] == "Bergen - Florida"] 

,index,Navn,Stasjon,Tid(norsk normaltid),Lufttemperatur,Vindretning,Middelvind,Nedbør (1 t)
0,0,Bergen - Florida,SN50540,01.01.2022 01:00,"2,8",279,"1,1",0
3,1,Bergen - Florida,SN50540,01.01.2022 02:00,3,92,"0,6",0
4,2,Bergen - Florida,SN50540,01.01.2022 03:00,"2,8",90,"1,8",0
6,3,Bergen - Florida,SN50540,01.01.2022 04:00,"2,1",141,"1,1",0
8,4,Bergen - Florida,SN50540,01.01.2022 05:00,"1,8",142,"0,9",0
...,...,...,...,...,...,...,...,...
47560,8754,Bergen - Florida,SN50540,31.12.2022 19:00,2,3,1,0
47562,8755,Bergen - Florida,SN50540,31.12.2022 20:00,"1,8",145,"0,9",0
47564,8756,Bergen - Florida,SN50540,31.12.2022 21:00,"1,6",263,2,0
47566,8757,Bergen - Florida,SN50540,31.12.2022 22:00,"0,6",281,"1,5","0,4"


### It seems like the data misses several hours - will probably be better to try API

